# W2D6 - 优化器与训练循环“八股文

**今日目标:**

1.  学会使用 `torch.optim` (优化器)，并理解 `model.parameters()` 的含义。
2.  **[重中之重]** 背诵并亲手默写出包含 5 个核心步骤的“训练循环八股文”。
3.  将本周所有知识点组合起来，运行你的第一个（伪）训练脚本。

## 最后一块拼图：优化器 (Optimizer)

`loss.backward()` 会帮我们计算出所有参数的**梯度 (`.grad`)**，但是，“计算出梯度”和“更新参数”是两回事。谁来执行“下山”的最后一步呢？

**答案：**优化器（Optimizer）

  * **职责：** 读取 `loss.backward()` 算好的梯度，然后根据一种特定的优化算法（如 SGD 或 Adam），将模型中所有参数**更新**掉。
  * **位置：** `torch.optim` 模块。
  * **常用算法：**
      * `torch.optim.SGD`：传统的随机梯度下降。
      * `torch.optim.Adam`：目前最常用、最鲁棒的优化器之一（通常是首选）。

**如何使用？**

**第一步：初始化 (在训练循环开始前)**
你必须告诉优化器它要“管理”哪些参数，以及“学习率 (lr)”是多少

In [ ]:
# 导入优化器
import torch.optim as optim
from utils import MyMLP, get_data_loaders
import torch.nn as nn

# 假设 model 是你在 W2D4 定义的 MyMLP 类的实例
model = MyMLP() 
criterion = nn.CrossEntropyLoss()

# 1. 告诉 Adam 优化器，它需要优化的参数是 model.parameters()
#    model.parameters() 是一个“魔法”方法，
#    它会自动返回你在 __init__ 中声明的所有可学习参数 (w 和 b)
# 2. lr=0.001 是学习率 (learning rate)
optimizer = optim.Adam(model.parameters(), lr=0.001) 

**关键点 (面试高频):**
`model.parameters()` 是 `nn.Module` 自动追踪参数能力的体现。这就是为什么[S1W2D4](day4.ipynb)强调**必须在 `__init__` 中用 `self.xxx` 来定义层**，否则 `model.parameters()` 会“看”不到它们，优化器也就无法更新它们！

**第二步：使用 (在训练循环内部)**
优化器有两个核心方法，你必须在循环中调用它们：

1.  **`optimizer.zero_grad()` (清空梯度)**

      * **作用：** 清空所有参数的 `.grad` 属性。
      * **为什么？** 正如 W2D3 所学，PyTorch 默认会**累加**梯度。我们必须在**下一次 `loss.backward()` 之前**手动清零，否则梯度会出错。

2.  **`optimizer.step()` (执行更新)**

      * **作用：** 优化器会遍历它管理的所有参数，然后用 `参数 = 参数 - 学习率 * 梯度` 的公式来更新它们。
      * **时机：** 必须在 `loss.backward()` 之后调用。

## “训练八股文”：The 5-Step Loop (必须背诵)


现在，我们把所有东西按顺序组装起来。这就是 PyTorch 训练的核心逻辑。

假设 `model`, `train_loader`, `criterion` (损失函数), `optimizer` 都已定义好。

In [ ]:
# 0. 将模型设置为“训练模式”
# 这会打开 Dropout 和 BatchNorm (如果模型里有的话)
model.train() 

# 遍历 DataLoader，每次取出一个批次 (batch) 的数据

train_loader, test_loader = get_data_loaders(batch_size=64, root='../../data')

for data_batch, labels_batch in train_loader:
    
    # [准备工作：处理数据形状]
    # 我们的 data_batch 是 [64, 1, 28, 28]
    # 但我们的 MLP (fc1) 需要 [64, 784]
    # 我们需要用 .view() 将其“压平”
    # -1 的意思是“自动计算这一维的大小” (在这里就是 64)
    data_batch = data_batch.view(-1, 784) 
    
    # -------------------------------------
    # --- “八股文” 5 个核心步骤开始 ---
    # -------------------------------------

    # 步骤 1：前向传播 (Forward Pass)
    # 将数据喂给模型，得到预测输出
    outputs = model(data_batch)

    # 步骤 2：计算损失 (Calculate Loss)
    # 用损失函数比较“预测输出”和“真实标签”
    loss = criterion(outputs, labels_batch)

    # 步骤 3：梯度清零 (Zero Gradients)
    # (在反向传播之前，清空上一轮的梯度)
    optimizer.zero_grad()

    # 步骤 4：反向传播 (Backward Pass)
    # 自动计算损失对所有可学习参数的梯度
    loss.backward()

    # 步骤 5：参数更新 (Update Parameters)
    # 优化器根据梯度，更新模型的权重
    optimizer.step()
    
    # -------------------------------------
    # --- “八股文” 5 个核心步骤结束 ---
    # -------------------------------------
    
    # (可选) 打印该批次的损失
    # .item() 是为了把只有一个元素的 Tensor 转换成 Python 数字
    print(f"当前批次的 Loss: {loss.item()}")

## W2D6 代码实践：组装！(The Full Script)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# --------------------------
# 1. W2D4: 定义模型 (nn.Module)
# --------------------------
class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(784, 128) # 784 = 28*28
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # 注意：W2D5 的 DataLoader 会给我们 [B, 1, 28, 28]
        # 我们需要在模型内部或外部处理它
        # 方案1: 在模型内部“压平” (推荐)
        x = x.view(-1, 784) # 将 [B, 1, 28, 28] 变为 [B, 784]
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# --------------------------
# 2. W2D5: 准备数据 (Dataset & DataLoader)
# --------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

# 我们只用一个小的子集来快速演示
# (实际训练时请删除下面这行)
train_subset = torch.utils.data.Subset(train_dataset, range(1024))

train_loader = DataLoader(
    dataset=train_subset, # (实际训练时用 train_dataset)
    batch_size=64,
    shuffle=True,
    num_workers=0 # 在 Windows 上 0 最安全，WSL/Linux 可以设 4
)

# --------------------------
# 3. W2D6: 定义“三大件”：模型实例、损失、优化器
# --------------------------
# (确保设备可用)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 实例化模型，并将其“搬到”GPU (如果可用)
model = MyMLP().to(device)

# 定义损失函数 (W2D2 知识点)
criterion = nn.CrossEntropyLoss()

# 定义优化器 (W2D6 知识点)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --------------------------
# 4. W2D6: 训练循环“八股文” (The Loop)
# --------------------------
print("\n--- 开始训练 ---")

# 设置一个 epoch (只训练一轮)
num_epochs = 1 

# 开启训练模式
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # 遍历 DataLoader
    for batch_idx, (data, labels) in enumerate(train_loader):
        
        # [准备] 将数据和标签也“搬到”GPU
        data = data.to(device)
        labels = labels.to(device)
        
        # [注意] 我们已在模型 forward 中处理了 .view()，这里不用再写
        
        # 1. 前向传播
        outputs = model(data)
        
        # 2. 计算损失
        loss = criterion(outputs, labels)
        
        # 3. 梯度清零
        optimizer.zero_grad()
        
        # 4. 反向传播
        loss.backward()
        
        # 5. 参数更新
        optimizer.step()
        
        # (可选) 打印损失
        if batch_idx % 10 == 0: # 每 10 个 batch 打印一次
            print(f"  Batch {batch_idx}/{len(train_loader)} - Loss: {loss.item():.4f}")

print("--- 训练完成 ---")

## **W2D6 今日行动清单**

1.  **✅ 运行代码：** 亲手将上面完整的“毕业代码”在你的环境中运行一遍。观察损失值（Loss）是如何随着训练（Batch 增加）而**逐渐变小**的。
2.  **✅ 理解形状（Debug）：** 重点理解 `x = x.view(-1, 784)` 这一行的**必要性**。这是新手最常卡住的地方（形状不匹配）。
3.  **✅ 默写 (核心！)：** **这是你今天的“成功标准”**。关掉所有参考资料，打开一个空白文件，尝试**从内存中默写出“八股文”的 5 个核心步骤**（`outputs = ...`, `loss = ...`, `zero_grad()`, `backward()`, `step()`）。
4.  **✅ 思考题 (面试模拟)：**
      * “我们已经有了 `model(data)`，为什么还要一个 `model.train()`？它和 `model.eval()` 有什么区别？”

你已经站在了 W3 项目实战的门口。完成今天的默写，你就真正掌握了 PyTorch 的核心。